In [ ]:
import numpy as np

from helper import * 
from IPython.display import clear_output
import matplotlib.pyplot as plt
import time
from scipy.linalg import expm
import random
import pickle

from zhinst.deviceutils.shfqa import SHFQA_SAMPLING_FREQUENCY
from zhinst.toolkit import SHFQAChannelMode, AveragingMode


interactive = 0

if interactive ==1: 
    %matplotlib widget
    figsize=(12,5)
    font_large=15
    font_medium=10
else:
    %matplotlib inline
    figsize=(24,10)
    font_large=30
    font_medium=20

# Setup
## Connect to device

In [ ]:
from zhinst.toolkit import Session

session = Session("localhost")
device = session.connect_device("DEV12073")

## Parameter definitions

In [ ]:
number_of_qubits = 4

# Setup parameters
## Readout Channel
qachannel_number = 0
qachannel_center_frequency = 7.1e9
qachannel_power_in = -50
qachannel_power_out = -30
max_amplitude_readout = 2.5 / number_of_qubits * 0.98

## Control Channels
sgchannel_number = [0, 1, 2, 3, 4]
sgchannel_center_frequency = [4.7e9, 4.7e9, 3.2e9, 3.2e9, 3e9]
sgchannel_power_out = [
    -25,
    -25,
    -25,
    -25,
    -25,
]
sgchannel_trigger_input = 0

## Qubit Parameters
qubit_params = {
    "readout_pulse_duration": 2e-6,
    "propagation_delay": 0,
    "qubit_readout_frequencies": [0] * number_of_qubits,
    "qubit_readout_widths": [0] * number_of_qubits,
    "qubit_readout_amplitudes": [0] * number_of_qubits,
    "max_drive_strength": [0] * number_of_qubits,
    "qubit_T1_time": [0] * number_of_qubits,
    "qubit_T2_time": [0] * number_of_qubits,
    "qubit_single_gate_time": [0] * number_of_qubits,
    "qubit_drive_frequency": [0] * number_of_qubits,
    "qubit_pi_amplitude": [0] * number_of_qubits,
    "qubit_pi/2_amplitude": [0] * number_of_qubits,
    "qubit_readout_rotation": [0] * number_of_qubits,
    "qubit_qscale": [-1] * number_of_qubits,
}

configure_device(
    device,
    number_of_qubits,
    qachannel_center_frequency,
    qachannel_power_in,
    qachannel_power_out,
    sgchannel_number,
    sgchannel_center_frequency,
    sgchannel_power_out,
    sgchannel_trigger_input,
)


# Resonator spectroscopy - CW

In [ ]:
sweeper = session.modules.shfqa_sweeper
sweeper.device(device)

sweeper.rf.channel(qachannel_number)
sweeper.rf.input_range(qachannel_power_in)
sweeper.rf.output_range(qachannel_power_out)
sweeper.rf.center_freq(qachannel_center_frequency)

sweeper.sweep.start_freq(-700e6)
sweeper.sweep.stop_freq(700e6)
sweeper.sweep.num_points(1001)
sweeper.sweep.mapping("linear")
sweeper.sweep.oscillator_gain(max_amplitude_readout)
sweeper.sweep.mode(True)

sweeper.average.integration_time(10000e-6) 
sweeper.average.num_averages(1)
sweeper.average.mode("cyclic")

enable_qa_channel(device,1)
wide_resonator_spectrum = sweeper.run()
enable_qa_channel(device,0)

In [ ]:
plot_spectroscopy(
    sweeper,
    wide_resonator_spectrum,
    fontsize=font_large,
    figsize=figsize,
    slope=0.54,
    saveloc="PSIMeasurements/largespec.png",
)

with open("PSIMeasurements/largespec.pkl", "wb") as f:
    pickle.dump(wide_resonator_spectrum, f)

# Resonator spectroscopy CW vs. Power

In [ ]:
qubit_params['qubit_readout_frequencies']=[125e6,402e6,-570e6,-157.5e6,-352e6]
qubit_params['qubit_readout_widths']=[20e6,20e6,20e6,20e6,20e6]
number_amplitude_values = 10;
average_factor=0.00000001;
resonator_spectrum_data={'qubits':[ [] for _ in range(number_of_qubits) ]}
relative_amplitude_values=np.linspace(max_amplitude_readout/number_amplitude_values,max_amplitude_readout,number_amplitude_values)

In [ ]:
enable_qa_channel(device,1)

for qubit in range(number_of_qubits):
    print('qubit:',qubit+1)
    sweeper.sweep.start_freq(qubit_params['qubit_readout_frequencies'][qubit]-qubit_params['qubit_readout_widths'][qubit])
    sweeper.sweep.stop_freq(qubit_params['qubit_readout_frequencies'][qubit]+qubit_params['qubit_readout_widths'][qubit])
    sweeper.sweep.num_points(101)
        
    for amplitude in relative_amplitude_values:
        sweeper.sweep.oscillator_gain(amplitude)
        sweeper.average.num_averages(int(np.ceil(average_factor*1/amplitude**2)))
        resonator_spectrum_data['qubits'][qubit].append(sweeper.run())
        clear_output(wait=True) # WHY?
        
enable_qa_channel(device,0)

In [ ]:
plot_spectroscopy_multiQubit(
    sweeper.sweep.num_points(),
    number_of_qubits,
    resonator_spectrum_data,
    relative_amplitude_values,
    font_large,
    font_medium,
    figsize,
    slope=0.54,
    saveloc="PSIMeasurements/indvspec",
)


with open("PSIMeasurements/indvspec.pkl", "wb") as f:
    pickle.dump(resonator_spectrum_data, f)

# Determine propagation delay

In [ ]:
qubit_params['qubit_readout_frequencies']=[407e6,130e6,-570e6,-157.5e6,-352e6]
qubit_params['qubit_readout_widths']=[5e6,5e6,5e6,5e6,5e6]
qubit_params['qubit_readout_amplitudes']=[max_amplitude_readout,max_amplitude_readout,max_amplitude_readout,max_amplitude_readout,max_amplitude_readout]

In [ ]:
envelope_duration = 1.0e-6
envelope_rise_fall_time = 0.05e-6
envelope_frequencies = [qubit_params["qubit_readout_frequencies"][0] + 100e6]
probe_pulse = generate_flat_top_gaussian(
    envelope_frequencies,
    envelope_duration,
    envelope_rise_fall_time,
    SHFQA_SAMPLING_FREQUENCY,
    scaling=0.95,
)

num_avg = 2 * 0
scope_trace = take_propagation_delay_measurement(
    session, device, qubit_params, num_avg, probe_pulse, envelope_duration
)

determine_propagation_delay(scope_trace, probe_pulse, qubit_params)

In [ ]:
fig3 = plt.figure(figsize=figsize)
plot_resonator_pulse_scope_trace(scope_trace)

# Qubit Spectroscopy

In [ ]:
qubit_params['propagation_delay']=226e-9

# define parameters
num_sweep_steps_qubit_spectroscopy = 1000
integration_time_qubit_spectroscopy = 20e-3

# preparation
min_max_frequencies=[[150,350e6],[150e6,350e6],[-50e6,00e6],[200e6,400e6],[25e6,100e6]]
qubit_params['max_drive_strength']=[1,1,0.5,0.5,0.5]

## setup SHFQC for qubit spectroscopy

In [ ]:
# generate readout pulses and weights
readout_pulse_duration_qubit_spectroscopy = 2e-6

readout_pulses = generate_flat_top_gaussian(
    frequencies=qubit_params["qubit_readout_frequencies"],
    pulse_duration=readout_pulse_duration_qubit_spectroscopy,
    rise_fall_time=5e-9,
    sampling_rate=SHFQA_SAMPLING_FREQUENCY,
    scaling=0.95,
)

weights = generate_integration_weights(readout_pulses)

# configure readout channel
device.qachannels[0].configure_channel(
    center_frequency=qachannel_center_frequency,
    input_range=qachannel_power_in,
    output_range=qachannel_power_out,
    mode=SHFQAChannelMode.READOUT,
)
upload_readout_waveforms(
    device, readout_pulses, weights, qubit_params, number_of_qubits
)

# configure readout channel
prepare_sequencer_and_resultLogger(
    device,
    integration_time_qubit_spectroscopy,
    readout_pulse_duration_qubit_spectroscopy,
    num_sweep_steps_qubit_spectroscopy,
    number_of_qubits,
)

# configure control channels
configure_control_sequences(
    device,
    number_of_qubits,
    min_max_frequencies,
    num_sweep_steps_qubit_spectroscopy,
    qubit_params,
    sgchannel_number,
)


## Measurements

In [ ]:
readout_results = run_experiment(
    device, sgchannel_number, number_of_qubits, reenable=False
)

In [ ]:
x_axis=[]
for qubit in range(number_of_qubits):
    x_axis.append(np.linspace(min_max_frequencies[qubit][0],min_max_frequencies[qubit][1],num_sweep_steps_qubit_spectroscopy)/10**6)
    
plot_multiQubit_results_AmpPhase(x_axis,readout_results,number_of_qubits,figsize,font_large,font_medium,saveloc='PSIMeasurements/qubitspec')


with open('PSIMeasurements/qubitspec.pkl', 'wb') as f:
    pickle.dump(readout_results, f)

# Pulsed Rabi Measurement - Determine Readout Angle and Pi-Pulse

In [ ]:
qubit_params['qubit_drive_frequency']=[299.260e6,275.990e6,-25e6,320e6,82e6]

# define parameters
num_steps_rabi_experiment = 100
num_averages_rabi_experiment = 2 ** 12
qubit_params["qubit_T1_time"] = [50e-6, 50e-6, 50e-6, 50e-6, 50e-6]
qubit_params["qubit_single_gate_time"] = [10e-9, 10e-9, 10e-9, 10e-9, 10e-9]
qubit_params["max_drive_strength"] = [1, 1, 1, 1, 1]

wait_factor_in_T1 = 5


In [ ]:
sgchannel_power_out = [10, 10, 10, 10, 10]

with device.set_transaction():
    for qubit in range(number_of_qubits):
        device.sgchannels[sgchannel_number[qubit]].output.range(
            sgchannel_power_out[qubit]
        )

In [ ]:
# generate and upload waveforms
readout_pulses_unscaled = generate_flat_top_gaussian(
    frequencies=qubit_params["qubit_readout_frequencies"],
    pulse_duration=qubit_params["readout_pulse_duration"],
    rise_fall_time=5e-9,
    sampling_rate=SHFQA_SAMPLING_FREQUENCY,
    scaling=0.99,
)

# configure result logger and weighted integration
weights = generate_integration_weights(readout_pulses_unscaled)

upload_readout_waveforms(
    device, readout_pulses_unscaled, weights, qubit_params, number_of_qubits
)

# rotate weights according to readout parameter settings
# weights = { i : np.multiply(weights[i], np.exp(1j * qubit_params['qubit_readout_rotation'][i])) for i in range(number_of_qubits)}

device.qachannels[0].readout.write_integration_weights(
    weights,
    # compensation for the delay between generator output and input of the integration unit
    integration_delay=qubit_params["propagation_delay"],
)

[
    single_qubit_pulse_time_samples,
    readout_pulse_duration_samples,
    pulse_startQA_string,
    weight_startQA_string,
] = prepare_sequencer_resultLogger_pulsedRabi(
    device,
    number_of_qubits,
    qubit_params,
    wait_factor_in_T1,
    num_averages_rabi_experiment,
    num_steps_rabi_experiment,
    singleShot=True,
)

configure_control_sequences_pulsedRabi(
    device,
    number_of_qubits,
    single_qubit_pulse_time_samples,
    qubit_params,
    num_averages_rabi_experiment,
    num_steps_rabi_experiment,
    sgchannel_number,
)

In [ ]:
device.qachannels[0].readout.configure_result_logger(
    result_source="integration",
    result_length=num_steps_rabi_experiment,
    num_averages=num_averages_rabi_experiment,
    averaging_mode=AveragingMode.CYCLIC,
)

readout_results = run_experiment(
    device, sgchannel_number, number_of_qubits, reenable=True
)

In [ ]:
x_axis = []
for qubit in range(number_of_qubits):
    x_axis = np.linspace(
        0, qubit_params["max_drive_strength"][qubit], num_steps_rabi_experiment
    )

plot_multiQubit_results_IQ(
    x_axis,
    readout_results,
    number_of_qubits,
    figsize,
    font_large,
    font_medium,
    saveloc="PSIMeasurements/Rabi1",
)


with open("PSIMeasurements/Rabi1.pkl", "wb") as f:
    pickle.dump(readout_results, f)

In [ ]:
qubit_params["qubit_readout_rotation"] = [
    -0.95 * np.pi,
    0.85 * np.pi,
    0.85 * np.pi,
    0.85 * np.pi,
    0.85 * np.pi,
]

# rotate weights according to readout parameter settings
weightsrot = {
    i: np.multiply(weights[i], np.exp(1j * qubit_params["qubit_readout_rotation"][i]))
    for i in range(number_of_qubits)
}

device.qachannels[0].readout.write_integration_weights(
    weightsrot,
    # compensation for the delay between generator output and input of the integration unit
    integration_delay=qubit_params["propagation_delay"],
)


device.qachannels[0].readout.configure_result_logger(
    result_source="integration",
    result_length=num_steps_rabi_experiment,
    num_averages=num_averages_rabi_experiment,
    averaging_mode=AveragingMode.CYCLIC,
)

readout_results = run_experiment(
    device, sgchannel_number, number_of_qubits, reenable=True
)

# plot results
x_axis = []
for qubit in range(number_of_qubits):
    x_axis = np.linspace(
        0, qubit_params["max_drive_strength"][qubit], num_steps_rabi_experiment
    )

plot_multiQubit_results_IQ(
    x_axis,
    readout_results,
    number_of_qubits,
    figsize,
    font_large,
    font_medium,
    saveloc="PSIMeasurements/Rabi2",
)


with open("PSIMeasurements/Rabi2.pkl", "wb") as f:
    pickle.dump(readout_results, f)


### fit data

In [ ]:
for qubit in range(number_of_qubits):
    amp_axis=np.linspace(0,qubit_params['max_drive_strength'][qubit],num_steps_rabi_experiment)
    y_data = np.abs(readout_results[qubit]-readout_results[qubit][0])
    if 0:
        y_data = amplitude_rabi(amp_axis, 45 , 5)
        noise = 1*np.random.normal(size=y_data.size)
        y_data += noise
    pars, stdevs=fit_data(amp_axis,y_data,amplitude_rabi,[30 , 10],1,figsize=figsize,font=font_medium,qubit=qubit,x_label='rel. qubit drive amplitude',y_label='amplitude [A.U.]',saveloc=f'PSIMeasurements/RabiFit{qubit}')
    print(pars)
    qubit_params['qubit_pi_amplitude'][qubit]= 2*np.pi/pars[0]
    qubit_params['qubit_pi/2_amplitude'][qubit]= np.pi/pars[0]
    
    print('qubit',qubit,': pi-amp. =',qubit_params['qubit_pi_amplitude'][qubit] ,'\n' 
          '          pi/2-amp. =', qubit_params['qubit_pi/2_amplitude'][qubit])

# Pulsed Ramsey Measurement - Determine qubit-frequency

In [ ]:
# define parameters
num_steps_ramsey_experiment = 100
num_averages_ramsey_experiment = 2**12
max_wait_time_target=10e-6
wait_factor_in_T1=5
ramsey_offset_frequency=0e6


wait_time_steps_samples=int(round(max_wait_time_target/(num_steps_ramsey_experiment-1)*SHFQA_SAMPLING_FREQUENCY/16)*16)
max_wait_time=(wait_time_steps_samples)*(num_steps_ramsey_experiment-1)*0.5e-9
print('max. wait time: ', round(max_wait_time *10**9*2)/2,' ns in steps of', round(wait_time_steps_samples*0.5*2)/2, 'ns')

In [ ]:
with device.set_transaction():
    for qubit in range(number_of_qubits):
        device.sgchannels[0].oscs[0].freq(qubit_params['qubit_drive_frequency'][qubit]+0.5e6)

## prepare SHFQC for Ramsey Measurements

In [ ]:
device.qachannels[0].readout.configure_result_logger(result_source='integration',result_length=num_steps_ramsey_experiment, num_averages=num_averages_ramsey_experiment)
device.qachannels[0].readout.run()

time_to_next_experiment_seqSamples=int(np.max(qubit_params['qubit_T1_time'])*wait_factor_in_T1*SHFQA_SAMPLING_FREQUENCY/8)

seqc_program_qa = f"""
var time_ind = 0;
waitDigTrigger(1); // wait for software trigger to start experiment
repeat({num_averages_ramsey_experiment}) {{
    for (time_ind = 0; time_ind < {num_steps_ramsey_experiment}; time_ind++) {{
        // Trigger control channels to play pulse
        setTrigger(1);         
        wait(1);
        setTrigger(0);
        
        playZero({wait_time_steps_samples}*time_ind+2*{single_qubit_pulse_time_samples});
        
        //start readout
        playZero({readout_pulse_duration_samples});
        startQA({pulse_startQA_string}, {weight_startQA_string}, true, 0, 0x0);
        
        //wait for qubit decay
        wait({time_to_next_experiment_seqSamples});
    }}
}}
"""

device.qachannels[0].generator.load_sequencer_program(seqc_program_qa)
device.qachannels[0].generator.enable_sequencer(single=True)

configure_control_sequences_pulsedRamsey(
    device,
    number_of_qubits,
    single_qubit_pulse_time_samples,
    qubit_params,
    num_averages_ramsey_experiment,
    num_steps_ramsey_experiment,
    wait_time_steps_samples,
    sgchannel_number,
)

In [ ]:
readout_results = run_experiment(
    device, sgchannel_number, number_of_qubits, reenable=False
)

In [ ]:
x_axis=[]
for qubit in range(number_of_qubits):
    x_axis=np.linspace(single_qubit_pulse_time_samples/SHFQA_SAMPLING_FREQUENCY,max_wait_time+single_qubit_pulse_time_samples/SHFQA_SAMPLING_FREQUENCY,num_steps_ramsey_experiment)*10**6

plot_multiQubit_results_IQ(x_axis,readout_results,number_of_qubits,figsize,font_large,font_medium,label='time between pulse centers [us]',saveloc='PSIMeasurements/Ramsey1')


with open('PSIMeasurements/Ramsey1.pkl', 'wb') as f:
    pickle.dump(readout_results, f)

### fit data

In [ ]:
update_qubit_parameters=0
startparam=[[0.5, 0.003,20,0*np.pi/2],[0.5, 0.00045,10,0*np.pi/2]]
offset=[0.0017,-0.002]
for qubit in range(number_of_qubits):
    time_axis=np.linspace(single_qubit_pulse_time_samples/SHFQA_SAMPLING_FREQUENCY,max_wait_time+single_qubit_pulse_time_samples/SHFQA_SAMPLING_FREQUENCY,num_steps_ramsey_experiment)*10**6
    y_data = np.imag(readout_results[qubit])+offset[qubit]
                     
    pars, stdevs=fit_data(time_axis,y_data,ramsey_time_exp,startparam[qubit],1,figsize=figsize,font=font_medium,qubit=qubit,x_label='time between pulse centers [us]',y_label='amplitude [A.U.]',saveloc=f'PSIMeasurements/Ramsey1fit{qubit}')    
    print(pars)
    if update_qubit_parameters: 
        qubit_params['qubit_drive_frequency'][qubit]+=pars[0]*10**6
        qubit_params['qubit_T2_time'][qubit]=pars[2]*10**-6
    print('qubit',qubit,': frequency =', qubit_params['qubit_drive_frequency'][qubit]+pars[0]*10**6, '\n', 
          '         T2 - time =', qubit_params['qubit_T2_time'][qubit])

# Qubit T1 Measurement

In [ ]:
# define parameters
num_steps_T1_experiment = 100
num_averages_T1_experiment = 2**12
max_wait_time_target=50e-6
wait_factor_in_T1=5

wait_time_steps_samples=int(round(max_wait_time_target/(num_steps_T1_experiment-1)*SHFQA_SAMPLING_FREQUENCY/16)*16)
max_wait_time=(wait_time_steps_samples)*(num_steps_T1_experiment-1)*0.5e-9
print('max. wait time: ', round(max_wait_time *10**9*2)/2,' ns in steps of', round(wait_time_steps_samples*0.5*2)/2, 'ns')

In [ ]:
device.qachannels[0].readout.configure_result_logger(result_source='integration',result_length=num_steps_T1_experiment, num_averages=num_averages_T1_experiment)
device.qachannels[0].readout.run()

time_to_next_experiment_seqSamples=int(np.max(qubit_params['qubit_T1_time'])*wait_factor_in_T1*SHFQA_SAMPLING_FREQUENCY/8)

seqc_program_qa = f"""
var time_ind = 0;
waitDigTrigger(1); // wait for software trigger to start experiment
repeat({num_averages_T1_experiment}) {{
    for (time_ind = 0; time_ind < {num_steps_T1_experiment}; time_ind++) {{
        // Trigger control channels to play pulse
        setTrigger(1);         
        wait(1);
        setTrigger(0);
        
        // wait until control sequence finished
        playZero({wait_time_steps_samples}*time_ind+{single_qubit_pulse_time_samples});
        
        //start readout
        playZero({readout_pulse_duration_samples});
        startQA({pulse_startQA_string}, {weight_startQA_string}, true, 0, 0x0);
        
        //wait for qubit decay
        wait({time_to_next_experiment_seqSamples});
    }}
}}

"""

device.qachannels[0].generator.load_sequencer_program(seqc_program_qa)
device.qachannels[0].generator.enable_sequencer(single=True)

configure_control_sequences_pulsedRamsey(
    device,
    number_of_qubits,
    single_qubit_pulse_time_samples,
    qubit_params,
    num_averages_T1_experiment,
    num_steps_T1_experiment,
    wait_time_steps_samples,
    sgchannel_number,
)

In [ ]:
readout_results = run_experiment(
    device, sgchannel_number, number_of_qubits, reenable=False
)

In [ ]:
x_axis=[]
for qubit in range(number_of_qubits):
    x_axis=np.linspace(single_qubit_pulse_time_samples/2/SHFQA_SAMPLING_FREQUENCY,max_wait_time+single_qubit_pulse_time_samples/2/SHFQA_SAMPLING_FREQUENCY,num_steps_T1_experiment)*10**6

plot_multiQubit_results_IQ(x_axis,readout_results,number_of_qubits,figsize,font_large,font_medium,label='time to readout pulse [us]',saveloc='PSIMeasurements/Decay1')


with open('PSIMeasurements/Decay1.pkl', 'wb') as f:
    pickle.dump(readout_results, f)

### fit data

In [ ]:
update_qubit_parameters=0

for qubit in range(number_of_qubits):
    time_axis=np.linspace(single_qubit_pulse_time_samples/2/SHFQA_SAMPLING_FREQUENCY,max_wait_time+single_qubit_pulse_time_samples/2/SHFQA_SAMPLING_FREQUENCY,num_steps_T1_experiment)*10**6
    y_data = np.abs(readout_results[qubit]-readout_results[qubit][-1])
    if 0:
        y_data = T1_time_exp(time_axis, 5, 24.5) # in us
        noise = 0.1*np.random.normal(size=y_data.size)
        y_data += noise
    pars, stdevs=fit_data(time_axis,y_data,T1_time_exp,[6,8],1,figsize=figsize,font=font_medium,qubit=qubit,x_label='time to readout [us]',y_label='amplitude [A.U.]',saveloc=f'PSIMeasurements/Decay1fit{qubit}')
    print(pars)
    if update_qubit_parameters: 
        qubit_params['qubit_T1_time'][qubit]=pars[1]*10**-6
    print('qubit',qubit,': T1 - time =', qubit_params['qubit_T1_time'][qubit] * 10**6, 'us')

# Randomized Benchmarking

In [ ]:
# n different sequence lengths
n_cliffords = 1
# k different random sequences per length
k_seeds = 1
# how many averages / repetitions of each sequence
num_Averages = 2**5
wait_factor_in_T1 = 1

In [ ]:
qubit = 0

pi_amplitude = qubit_params["qubit_pi_amplitude"][qubit]
pi_length = qubit_params["qubit_single_gate_time"][qubit]
pi2_amplitude = qubit_params["qubit_pi/2_amplitude"][qubit]
pi2_length = pi_length
qscale = -qubit_params["qubit_qscale"][qubit]*5

In [ ]:
pulse = pulse_envelope(1, 50e-9, 0, qscale=-10)
plt.plot(np.real(pulse))
plt.plot(np.imag(pulse))
plt.plot(np.square(np.abs(np.real(pulse)+np.imag(pulse))))
plt.plot(0*np.real(pulse))

#### build the clifford gates out of the elementary single qubit gates

In [ ]:
# all elements of the Clifford group, according to defintion in arXiv:1410.2338
clifford_params = [
    ['I'],
    ['Y/2', 'X/2'],
    ['-X/2', '-Y/2'],
    ['X'],
    ['-Y/2', '-X/2'],
    ['X/2', '-Y/2'],
    ['Y'],
    ['-Y/2', 'X/2'],
    ['X/2', 'Y/2'],
    ['X', 'Y'],
    ['Y/2', '-X/2'],
    ['-X/2', 'Y/2'],
    ['Y/2', 'X'],
    ['-X/2'],
    ['X/2', '-Y/2', '-X/2'],
    ['-Y/2'],
    ['X/2'],
    ['X/2', 'Y/2', 'X/2'],
    ['-Y/2', 'X'],
    ['X/2', 'Y'],
    ['X/2', '-Y/2', 'X/2'],
    ['Y/2'],
    ['-X/2', 'Y'],
    ['X/2', 'Y/2', '-X/2']
]

clifford_len = len(clifford_params)

# parameters of basic single qubit pulses
pulses_params = {
    'I': {'amplitude':0.0, 'length': pi_length, 'phase': 0.0, 'qscale': qscale},
    'X': {'amplitude':pi_amplitude, 'length': pi_length, 'phase': 0.0, 'qscale': qscale},
    'Y': {'amplitude':pi_amplitude, 'length': pi_length, 'phase': 90.0, 'qscale': qscale},
    'X/2': {'amplitude':pi2_amplitude, 'length': pi2_length, 'phase': 0.0, 'qscale': qscale},
    'Y/2': {'amplitude':pi2_amplitude, 'length': pi2_length, 'phase': 90.0, 'qscale': qscale},
    '-X/2': {'amplitude':pi2_amplitude, 'length': pi2_length, 'phase': 0.0-180.0, 'qscale': qscale},
    '-Y/2': {'amplitude':pi2_amplitude, 'length': pi2_length, 'phase': 90.0-180.0, 'qscale': qscale},
}

# calculate complex waveforms for single qubit elementary pulses
pulses_waves = {pulse_type: pulse_envelope(**pulse_param) for (pulse_type, pulse_param) in pulses_params.items()}
# calculate complex waveforms for each of the Clifford gates
clifford_waves = [np.concatenate([pulses_waves[i] for i in clifford_gate]) for clifford_gate in clifford_params]

#### basic definitions to manipulate Clifford gates - needed for recovery gate calculation

In [ ]:
def pauli(ind = 'x'):
    """pauli matrices
    """
    if ind =='x':
        res = np.array([[0,1], [1,0]])
    if ind =='y':
        res = np.array([[0,-1j], [1j,0]])
    if ind =='z':
        res = np.array([[1,0], [0,-1]])
        
    return res

def rot_matrix(angle=np.pi, axis='x'):
    """general definition of rotation unitary for a single qubit
    """
    return expm(-1j * angle / 2 * pauli(axis))

def mult_gates(gates, use_linalg=False, tol=20):
    """multiply a variable number of gates / matrices - recursive definition faster for simple 2x2 matrices
    """
    if len(gates) > 1:
        if use_linalg:
            res = np.linalg.multi_dot(gates)
        else:
            res = np.matmul(gates[0], mult_gates(gates[1:], use_linalg=False, tol=tol))
    elif len(gates) == 1:
        res = gates[0]
    
    return res.round(tol)

# generate matrix representation of all Clifford gates from elementary gates
elem_gates = {'I': np.array([[1,0],[0,1]]),
              'X': rot_matrix(np.pi, 'x'),
              'Y': rot_matrix(np.pi, 'y'),
              'X/2': rot_matrix(np.pi / 2, 'x'),
              'Y/2': rot_matrix(np.pi / 2, 'y'),
              '-X/2': rot_matrix(-np.pi / 2, 'x'),
              '-Y/2': rot_matrix(-np.pi / 2, 'y')}

clifford_matrices = [[elem_gates[gate] for gate in gates] for gates in clifford_params]
clifford_gates = [mult_gates(matrices) for matrices in clifford_matrices]

def glob_phase(phase, dim=2):
    """global phase operator for dimensionality dim
    """
    return np.exp(1j * phase) * np.identity(dim)

def match_up_to_phase(target, gates, dim=2, verbose=False):
    """finds the element of the list gates that best matches the target gate up to a global phase of integer multiples of pi
    """
    # set of global phase operators for integer multiples of pi
    glob_phases = [glob_phase(0, dim), glob_phase(np.pi, dim)]
    # gates up to global phases
    gates_2 = [[mult_gates([gate1, gate2]) for gate2 in glob_phases] for gate1 in gates]
    # index of gate that is closest to target up to global phase (using frobenius norm)
    match_index = np.argmin([np.amin([np.linalg.norm(target - gate) for gate in gates]) for gates in gates_2])
    
    return match_index

#### function to calculate the last gate in the sequence - recovery gate which leads back to initial state (up to global phase)

In [ ]:
def calculate_inverse_clifford(clifford_list, gates=clifford_gates):
    """Calculates the final recovery clifford gate

    Parameters:
    clifford_list: list
        a list containing the indices of the clifford sequence to be inverted
    gate: list
        a list containing the gates to compare the recovery gate to
    """
    # matrix representation of Clifford sequence
    seq_gate = mult_gates([gates[gate] for gate in clifford_list])
    # recovery gate - inverse of full sequence
    rec_gate = np.linalg.inv(seq_gate)
    # index of recovery gate (up to global phase)
    recovery = int(match_up_to_phase(rec_gate, clifford_gates))
    
    return recovery

#### A computational representation of the PRNG on the HDAWG

In [ ]:
class HDAWG_PRNG:
    def __init__(self, seed=0xcafe, lower=0, upper=2**16-1):
        self.lsfr = seed
        self.lower = lower
        self.upper = upper

    def next(self):
        lsb = self.lsfr & 1
        self.lsfr = self.lsfr >> 1
        if (lsb):
            self.lsfr = 0xb400 ^ self.lsfr
        rand = ((self.lsfr * (self.upper-self.lower+1) >> 16) + self.lower) & 0xffff
        return rand

## Prepare the SHFQC for RB sequence

In [ ]:
# configured in single shot mode
singleshot = False
if singleshot:
    exp = num_Averages
    avg = 1
else:
    exp = 1
    avg = num_Averages

device.qachannels[0].readout.configure_result_logger(result_source='integration',result_length=exp*k_seeds, num_averages=avg)
device.qachannels[0].readout.run()

time_to_next_experiment_seqSamples=int(np.max(qubit_params['qubit_T1_time'])*wait_factor_in_T1*SHFQA_SAMPLING_FREQUENCY/8)

seqc_program_qa = f"""
waitDigTrigger(1);
setTrigger(1);
wait(10);
setTrigger(0);

repeat({exp*avg*k_seeds})
{{

    waitDigTrigger(2);

    //start readout
    
    playZero({readout_pulse_duration_samples});
    startQA({pulse_startQA_string}, {weight_startQA_string}, true, 0, 0x0);
    
}}

"""
device.qachannels[0].generator.load_sequencer_program(seqc_program_qa)
device.qachannels[0].generator.enable_sequencer(single=True)

configure_control_sequences_pulsedRB(
    device,
    number_of_qubits,
    clifford_waves,
    clifford_len,
    qubit_params,
    num_Averages,
    sgchannel_number,
)

In [ ]:
AWG_REGISTER_M1 = 0
AWG_REGISTER_SEED = 1
AWG_REGISTER_RECOVERY = 2

In [ ]:
M = n_cliffords

# generate random seed for the PRNG
seed = random.randrange(1, 2 ** 16 - 1)
# use seed and PRNG model to determine gate sequence that will be played on AWG
prng = HDAWG_PRNG(seed=seed, lower=0, upper=clifford_len - 1)
gates_M1 = [prng.next() for i in range(M)]
# calculate the recovery clifford gate
gate_M = calculate_inverse_clifford(gates_M1)

registers = [
    (AWG_REGISTER_M1, M),
    (AWG_REGISTER_SEED, seed),
    (AWG_REGISTER_RECOVERY, gate_M),
]

with device.set_transaction():
    for register, value in registers:
        device.sgchannels[sgchannel_number[qubit]].awg.userregs[register](value)

readout_results = run_experiment(
    device, sgchannel_number, number_of_qubits=1, reenable=True
)


In [ ]:
n_start = 2

# set the AWG to a known state
#awg.stop()

start = time.perf_counter_ns()
upload_time = []
exec_time = []


for len_exp in range(n_start, n_cliffords + n_start):
    # length of sequence is 2**n
    M = 2**len_exp

    # iterate over different random sequences
    for rand_i in range(k_seeds):

        exec_start = time.perf_counter_ns()

        # generate random seed for the PRNG
        seed = random.randrange(1, 2**16-1)
        # use seed and PRNG model to determine gate sequence that will be played on AWG
        prng = HDAWG_PRNG(seed=seed,lower=0,upper=clifford_len-1)
        gates_M1 = [prng.next() for i in range(M)]
        # calculate the recovery clifford gate
        gate_M = calculate_inverse_clifford(gates_M1)

        up_start = time.perf_counter_ns()

        # set AWG registers to transmit information on 
        # - sequence length
        # - PRNG seed
        # - last clifford gate in sequence
        registers = [
            (AWG_REGISTER_M1, M),
            (AWG_REGISTER_SEED, seed),
            (AWG_REGISTER_RECOVERY, gate_M)
        ]

        with device.set_transaction():
            for register, value in registers:
                device.sgchannels[sgchannel_number[qubit]].awg.userregs[register](value)

        upload_time.append(time.perf_counter_ns() - up_start)

        run_experiment(device, sgchannel_number, number_of_qubits=1, reenable=True)

        exec_time.append(time.perf_counter_ns() - exec_start)

upload_time = np.array(upload_time) * 1e-9
exec_time = np.array(exec_time) * 1e-9

tot_time = time.perf_counter_ns() - start
print(f'Total time: {tot_time*1e-9:.3f}s')
print(f'Per iteration time: {tot_time/(k*n)*1e-9:.3f}s')